In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import imageio.v2 as iio
import cv2
# import open3d as o3d
import torch

In [ ]:
def get_bounds(xyz):
    min_xyz = np.min(xyz, axis=0)
    max_xyz = np.max(xyz, axis=0)
    min_xyz -= 0.05
    max_xyz += 0.05
    bounds = np.stack([min_xyz, max_xyz], axis=0)
    bounds = bounds.astype(np.float32)
    return bounds

def convert_bound_2to8(bound):
    start, end=bound
    x = [start[0], end[0], end[0], start[0], start[0], end[0], end[0], start[0]]
    y = [start[1], start[1], end[1], end[1], start[1], start[1], end[1], end[1]]
    z = [start[2], start[2], start[2], start[2], end[2], end[2], end[2], end[2]]
    return np.array(list(zip(x, y, z)))

def bound2ret(bound, ax):
    corners = convert_bound_2to8(bound)

    # 绘制长方体的边
    edges = [(0, 1), (1, 2), (2, 3), (3, 0), (4, 5), (5, 6), (6, 7), (7, 4), (0, 4), (1, 5), (2, 6), (3, 7)]
    for edge in edges:
        ax.plot([corners[edge[0]][0], corners[edge[1]][0]], 
                [corners[edge[0]][1], corners[edge[1]][1]], 
                [corners[edge[0]][2], corners[edge[1]][2]], 'b-')

In [ ]:
debug_root='debug/beat_matching/'

matching_items = ['mask_at_box']
ohter = 'test-zjumocap'

mask_at_box_beats = np.load(f'{debug_root}mask_at_box-beats.npy').reshape(512,512)
mask_at_box_ohter = np.load(f'{debug_root}mask_at_box-{ohter}.npy').reshape(512,512)

print(mask_at_box_beats.shape)
print(mask_at_box_ohter.shape)

In [ ]:
data_path = 'debug/youtube/'
# data_path = 'debug/h36m-small/'
bounds = np.load(f'{data_path}bounds.npy')
tpose = np.load(f'{data_path}tpose.npy')
batch_ppts = np.load(f'{data_path}batch_ppts.npy')
batch_wpts = np.load(f'{data_path}batch_wpts.npy')
batch_wbounds = np.load(f'{data_path}batch_wbounds.npy')
batch_ray_o=np.load(f'{data_path}batch_ray_o.npy')
batch_ray_d=np.load(f'{data_path}batch_ray_d.npy')
batch_near=np.load(f'{data_path}batch_near.npy')
batch_far=np.load(f'{data_path}batch_far.npy')

# wpts = np.load(f'{data_path}wpts.npy')
# ray_o = np.load(f'{data_path}ray_o.npy')
# ray_d = np.load(f'{data_path}ray_d.npy')
# nerf_near = np.load(f'{data_path}nerf_near.npy')
# nerf_far = np.load(f'{data_path}nerf_far.npy')

mask_at_box = np.load(f'{data_path}mask_at_box.npy')
frame_index = np.load(f'{data_path}frame_index.npy')

In [ ]:
print(frame_index)

In [ ]:
print(mask_at_box.min())
mask_at_box_len = np.sqrt(mask_at_box.shape[0]).astype(np.int32)
mask_at_box_img = mask_at_box.reshape(mask_at_box_len, mask_at_box_len)
print(mask_at_box_img.shape)
plt.imshow(mask_at_box_img)
plt.show()

In [ ]:
print(batch_wpts.shape)
print(wpts.shape)

In [ ]:
# tpose_path = 'data/zju-mocap/my_377/smpl_lbs/bigpose_vertices.npy'
# tpose_path = 'data/test-zjumocap/smpl_lbs/bigpose_vertices.npy'
# tpose_path = 'tools/S9tpose.npy'
# tpose_path = 'data/h36m/small/smpl_lbs/bigpose_vertices.npy'
# tpose_path = 'data/h36m/S9/smpl_lbs/bigpose_vertices.npy'
# tpose = np.load(tpose_path)

In [ ]:
batch_nerf_near_pts=batch_ray_o+batch_ray_d[:,:]*batch_near[:, None]
batch_nerf_far_pts=batch_ray_o+batch_ray_d[:,:]*batch_far[:, None]
print(batch_nerf_near_pts.shape)
print(batch_nerf_far_pts.shape)

In [ ]:
nerf_near_pts=ray_o+ray_d*nerf_near[:, :, None]
nerf_far_pts=ray_o+ray_d*nerf_far[:, :, None]
print(nerf_near_pts.shape)
print(nerf_far_pts.shape)

In [ ]:
# 创建3D图形
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# world points #

# sample
# ax.scatter(w_pts[:, 0], w_pts[:, 1], w_pts[:, 2], c='tab:red', marker='o')

# near sample
# ax.scatter(nerf_near_pts[:, :, 0], nerf_near_pts[:, :, 1], nerf_near_pts[:, :, 2], c='tab:blue', marker='.')

# far sample
# ax.scatter(nerf_far_pts[:, :, 0], nerf_far_pts[:, :, 1], nerf_far_pts[:, :, 2], c='tab:green', marker='.')

# near sample
ax.scatter(batch_nerf_near_pts[:, 0], batch_nerf_near_pts[:, 1], batch_nerf_near_pts[:, 2], c='tab:red', marker='.', alpha=0.1)

# # far sample
ax.scatter(batch_nerf_far_pts[:, 0], batch_nerf_far_pts[:, 1], batch_nerf_far_pts[:, 2], c='tab:green', marker='.', alpha=0.1)

# world points
ax.scatter(batch_wpts[:, 0], batch_wpts[:, 1], batch_wpts[:, 2], c='tab:blue', marker='o')

# ray_o
# ax.scatter(O[0], O[1], O[2], c='green', marker='1', s=500)

# world bounds
# ax.scatter(batch_wbounds[:, 0], batch_wbounds[:, 1], batch_wbounds[:, 2], c='black', marker='^')

# pose points #
# tpose points
# ax.scatter(tpose[:, 0], tpose[:, 1], tpose[:, 2], c='tab:blue', marker='o')
# ax.scatter(tpose_bounds[:, 0], tpose_bounds[:, 1], tpose_bounds[:, 2], c='black', marker='^')

# pose points
# ax.scatter(batch_ppts[:, 0], batch_ppts[:, 1], batch_ppts[:, 2], c='tab:blue', marker='o')

# pbounds
# ax.scatter(batch_pbounds[:, 0], batch_pbounds[:, 1], batch_pbounds[:, 2], c='black', marker='^')

# sample pose points
# ax.scatter(p_pts[:, 0], p_pts[:, 1], p_pts[:, 2], c='tab:red', marker='o')

# 绘制pts_bounds
# ax.scatter(pts_bounds[:, 0], pts_bounds[:, 1], pts_bounds[:, 2], c='r', marker='o')

# 绘制pbounds
# ax.scatter(p_bounds[:, 0], p_bounds[:, 1], p_bounds[:, 2], c='b', marker='^')

# for label, part_bound in bbox.items():
#     if label != 'body' : continue
#     verts=convert_bound_2to8(part_bound)
#     ax.scatter(verts[:,0], verts[:,1], verts[:,2], c='green', marker='o')

# for i in range(5):
#     if i!=0: continue
#     verts=convert_bound_2to8(extra_bounds[i])
#     ax.scatter(verts[:,0], verts[:,1], verts[:,2], c='red', marker='o')
#     break


# for label, (start, end) in bbox.items():
#     ax.scatter(start[0], start[1], start[2], c='r', marker='o')
#     ax.scatter(end[0], end[1], end[2], c='r', marker='o')

bound2ret(batch_wbounds, ax)

# i=0
# part_bounds = [bounds[0], bounds[1]]
# for bound in part_bounds:
#     bound2ret(bound, ax)

# ax.view_init(-90, 90)
# ax.view_init(0, 0)
# ax.view_init(0, 90)
# 设置坐标轴标签
ax.set_aspect('equal')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

# 显示图形
plt.show()